# OptiGuide Example



Here we give a "zero-shot" learning example.

In [1]:
# %%capture # no printing logs here.
# # Install Gurobi for optimization
# %pip install gurobipy
# %pip install eventlet

# # Install FLAML for agents
# %pip install "flaml[optiguide]"

In [2]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

import re
import requests  # for loading the example source code


In [3]:
from flaml.autogen.agent import Agent
from flaml.autogen.agent.opti_guide import OptiGuideAgent


from flaml import oai
config_list = oai.config_list_gpt4_gpt35()
oai.ChatCompletion.start_logging()



In [4]:
import openai
import os

openai.api_type = "azure"
openai.api_base = "https://msrcore.openai.azure.com"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("CORE_AZURE_KEY").strip().rstrip()

In [5]:
# Load the Coffee example
code = """

import time

from gurobipy import GRB, Model

# Example data

capacity_in_supplier = {'supplier1': 150, 'supplier2': 50, 'supplier3': 100}

shipping_cost_from_supplier_to_roastery = {
    ('supplier1', 'roastery1'): 5,
    ('supplier1', 'roastery2'): 4,
    ('supplier2', 'roastery1'): 6,
    ('supplier2', 'roastery2'): 3,
    ('supplier3', 'roastery1'): 2,
    ('supplier3', 'roastery2'): 7
}

roasting_cost_light = {'roastery1': 3, 'roastery2': 5}

roasting_cost_dark = {'roastery1': 5, 'roastery2': 6}

shipping_cost_from_roastery_to_cafe = {
    ('roastery1', 'cafe1'): 5,
    ('roastery1', 'cafe2'): 3,
    ('roastery1', 'cafe3'): 6,
    ('roastery2', 'cafe1'): 4,
    ('roastery2', 'cafe2'): 5,
    ('roastery2', 'cafe3'): 2
}

light_coffee_needed_for_cafe = {'cafe1': 20, 'cafe2': 30, 'cafe3': 40}

dark_coffee_needed_for_cafe = {'cafe1': 20, 'cafe2': 20, 'cafe3': 100}

cafes = list(set(i[1] for i in shipping_cost_from_roastery_to_cafe.keys()))
roasteries = list(
    set(i[1] for i in shipping_cost_from_supplier_to_roastery.keys()))
suppliers = list(
    set(i[0] for i in shipping_cost_from_supplier_to_roastery.keys()))


# Create a new model
model = Model("coffee_distribution")
m = model

# OPTIGUIDE DATA CODE GOES HERE


# Create variables
x = model.addVars(shipping_cost_from_supplier_to_roastery.keys(),
                  vtype=GRB.INTEGER,
                  name="x")
y_light = model.addVars(shipping_cost_from_roastery_to_cafe.keys(),
                        vtype=GRB.INTEGER,
                        name="y_light")
y_dark = model.addVars(shipping_cost_from_roastery_to_cafe.keys(),
                       vtype=GRB.INTEGER,
                       name="y_dark")

# Set objective
model.setObjective(
    sum(x[i] * shipping_cost_from_supplier_to_roastery[i]
        for i in shipping_cost_from_supplier_to_roastery.keys()) +
    sum(roasting_cost_light[r] * y_light[r, c] +
        roasting_cost_dark[r] * y_dark[r, c]
        for r, c in shipping_cost_from_roastery_to_cafe.keys()) + sum(
            (y_light[j] + y_dark[j]) * shipping_cost_from_roastery_to_cafe[j]
            for j in shipping_cost_from_roastery_to_cafe.keys()), GRB.MINIMIZE)

# Conservation of flow constraint
for r in set(i[1] for i in shipping_cost_from_supplier_to_roastery.keys()):
    model.addConstr(
        sum(x[i]
            for i in shipping_cost_from_supplier_to_roastery.keys()
            if i[1] == r) == sum(
                y_light[j] + y_dark[j]
                for j in shipping_cost_from_roastery_to_cafe.keys()
                if j[0] == r), f"flow_{r}")

# Add supply constraints
for s in set(i[0] for i in shipping_cost_from_supplier_to_roastery.keys()):
    model.addConstr(
        sum(x[i]
            for i in shipping_cost_from_supplier_to_roastery.keys()
            if i[0] == s) <= capacity_in_supplier[s], f"supply_{s}")

# Add demand constraints
for c in set(i[1] for i in shipping_cost_from_roastery_to_cafe.keys()):
    model.addConstr(
        sum(y_light[j]
            for j in shipping_cost_from_roastery_to_cafe.keys()
            if j[1] == c) >= light_coffee_needed_for_cafe[c],
        f"light_demand_{c}")
    model.addConstr(
        sum(y_dark[j]
            for j in shipping_cost_from_roastery_to_cafe.keys()
            if j[1] == c) >= dark_coffee_needed_for_cafe[c], f"dark_demand_{c}")

# Optimize model
model.optimize()

# OPTIGUIDE CONSTRAINT CODE GOES HERE

# Solve
m.update()
model.optimize()

print(time.ctime())
if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
else:
    print("Not solved to optimality. Optimization status:", m.status)

"""

In [6]:
example_qa = """
----------
Question: Why is it not recommended to use just one supplier for roastery 2?
Answer Code:
```python
z = m.addVars(suppliers, vtype=GRB.BINARY, name="z")
m.addConstr(sum(z[s] for s in suppliers) <= 1, "_")
for s in suppliers:
    m.addConstr(x[s,'roastery2'] <= capacity_in_supplier[s] * z[s], "_")
```

----------
Question: What if there's a 13% jump in the demand for light coffee at cafe1?
Answer Code:
```python
light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
```

"""

In [7]:
agent = OptiGuideAgent(name="OptiGuide Coffee Example", 
                  source_code=code,
#                   doc_str="", 
                  example_qa="")

user = Agent("user", system_message="")
# user.organize_memory(agent)

Set parameter TokenServer to value "ishai-z420"
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 64 physical cores, 128 logical processors, using up to 32 threads
Optimize a model with 11 rows, 18 columns and 36 nonzeros
Model fingerprint: 0xd4254685
Variable types: 0 continuous, 18 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 3490.0000000
Presolve time: 0.00s
Presolved: 11 rows, 18 columns, 36 nonzeros
Variable types: 0 continuous, 18 integer (0 binary)
Found heuristic solution: objective 3486.0000000

Root relaxation: objective 2.470000e+03, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2470.0000000 2470.00000  0.0

In [8]:
user.send("What if we prohibit shipping from supplier 1 to roastery 2?", agent)


user (to OptiGuide Coffee Example):

What if we prohibit shipping from supplier 1 to roastery 2?

--------------------------------------------------------------------------------
coder interpreter (to pencil):


Question: What if we prohibit shipping from supplier 1 to roastery 2?
Answer Code:


--------------------------------------------------------------------------------
pencil (to coder interpreter):

# Prohibit shipping from supplier 1 to roastery 2
model.addConstr(x[('supplier1', 'roastery2')] == 0, "No_Shipping_S1R2")

# Solve after the new constraint
model.update()
model.optimize()

--------------------------------------------------------------------------------
# Prohibit shipping from supplier 1 to roastery 2
model.addConstr(x[('supplier1', 'roastery2')] == 0, "No_Shipping_S1R2")

# Solve after the new constraint
model.update()
model.optimize()
safeguard (to shield):


--- Code ---
# Prohibit shipping from supplier 1 to roastery 2
model.addConstr(x[('supplier1', 'roastery2')

In [9]:
user.send("What is the impact of supplier1 being able to supply only half the quantity at present?", agent)


user (to OptiGuide Coffee Example):

What is the impact of supplier1 being able to supply only half the quantity at present?

--------------------------------------------------------------------------------
coder interpreter (to pencil):


Question: What is the impact of supplier1 being able to supply only half the quantity at present?
Answer Code:


--------------------------------------------------------------------------------
pencil (to coder interpreter):

To examine the impact of supplier1 being able to supply only half the quantity, we can update the capacity in the `capacity_in_supplier` dictionary for supplier1 and reoptimize the model.

Here's the code snippet to update the capacity of supplier1:

```python
capacity_in_supplier['supplier1'] = capacity_in_supplier['supplier1'] // 2
model.update()

model.optimize()
```

Add this code snippet to the lines with substring "# OPTIGUIDE *** CODE GOES HERE" and check the optimization result to see the impact on the objective value.

